<a href="https://colab.research.google.com/github/ChicagoPark/chicago_work/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary Modules
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
from keras.preprocessing.image import ImageDataGenerator # Data Augmentation(Image transformations) 을 하기 위해 필요한 모듈

import scipy
import keras
from tensorflow.keras.utils import plot_model
! pip install keras_tuner
import keras_tuner as kt
import shutil

In [7]:
# Colab 상의 훈련용 데이터 디렉터리 정리
!mkdir training_data
%cd training_data/
!mkdir '0. stone'
!mkdir '1. pothole'
!mkdir '2. filled'
%cd ..

mkdir: cannot create directory ‘training_data’: File exists
/content/training_data
mkdir: cannot create directory ‘0. stone’: File exists
mkdir: cannot create directory ‘1. pothole’: File exists
mkdir: cannot create directory ‘2. filled’: File exists
/content


In [16]:
'''
# data path
# 인터넷에서 주워온것임 ! gdown --id 1PGhXUnaJDpcjgoLEhNn9gRvGZGRCzWUt
'''

# 빅데이터 AI 수업에서 공유된 data.zip 
#! gdown --id 1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO --out 'data.zip'

# 빅데이터 AI 수업에서 공유된 실차 데이터.zip
! gdown --id 1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L --out '실차데이터.zip'

# 직접 데이터 추출 및 작업을 진행한 구글드라이브 상 데이터
# ! gdown --id <추가예정> --out '<설정하고자하는폴더명>.zip'

!unzip '실차데이터.zip' -d '실차데이터'

Downloading...
From: https://drive.google.com/uc?id=1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L
To: /content/실차데이터.zip
100% 462M/462M [00:02<00:00, 178MB/s]


In [21]:
# 위 코드로 training_data 의 0. stone 경로에 데이터를 가지고 온다.
!mv /content/실차데이터/'0. no_pothole'/* /content/training_data/'0. stone'

In [ ]:
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/data/2. filled/V0F_HY_2332_20201223_163544_E_CH1_Busan_Sun_Mainroad_Day_23637.png  
  inflating: data/data/2. filled/V0F_HY_2332_20210104_105139_N_CH2_Busan_Sun_Frontback_Day_42196.png  
  inflating: data/data/2. filled/V0F_HY_2335_20201223_144538_E_CH2_Busan_Sun_Mainroad_Day_60292.png  
  inflating: data/data/2. filled/V0F_HY_2338_20201223_140718_E_CH2_Busan_Sun_Frontback_Day_57135.png  
  inflating: data/data/2. filled/V0F_HY_2338_20210115_081337_N_CH1_Busan_Sun_Mainroad_Sunset_07851.png  
  inflating: data/data/2. filled/V0F_HY_2340_20201223_160052_E_CH1_Busan_Sun_Frontback_Day_40761.png  
  inflating: data/data/2. filled/V0F_HY_2341_20201122_114932_N_CH0_Busan_Rain_Mainroad_Day_52956.png  
  inflating: data/data/2. filled/V0F_HY_2342_20201117_140231_E_CH1_Busan_Sun_Mainroad_Day_29401.png  
  inflating: data/data/2. filled/V0F_HY_2343_20201224_081055_N_CH2_Busan_Cloud_Mainroad_Day_43320.png  
  inflating: data/data/2. filled/V0F_

Downloading...
From: https://drive.google.com/uc?id=1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L
To: /content/__ ___.zip
100% 462M/462M [00:02<00:00, 176MB/s]


In [ ]:
'''data
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data
'''
! unzip 실차데이터.zip -d 실차데이터1

Archive:  실차데이터.zip
  inflating: 실차데이터1/0. no_pothole/100490.png  
  inflating: 실차데이터1/0. no_pothole/100880.png  
  inflating: 실차데이터1/0. no_pothole/101490.png  
  inflating: 실차데이터1/0. no_pothole/10170.png  
  inflating: 실차데이터1/0. no_pothole/102100.png  
  inflating: 실차데이터1/0. no_pothole/102450.png  
  inflating: 실차데이터1/0. no_pothole/102810.png  
  inflating: 실차데이터1/0. no_pothole/103340.png  
  inflating: 실차데이터1/0. no_pothole/103800.png  
  inflating: 실차데이터1/0. no_pothole/104080.png  
  inflating: 실차데이터1/0. no_pothole/104700.png  
  inflating: 실차데이터1/0. no_pothole/105440.png  
  inflating: 실차데이터1/0. no_pothole/105850.png  
  inflating: 실차데이터1/0. no_pothole/10620.png  
  inflating: 실차데이터1/0. no_pothole/107790.png  
  inflating: 실차데이터1/0. no_pothole/107990.png  
  inflating: 실차데이터1/0. no_pothole/108180.png  
  inflating: 실차데이터1/0. no_pothole/108490.png  
  inflating: 실차데이터1/0. no_pothole/108780.png  
  inflating: 실차데이터1/0. no_pothole/1130.png  
  inflating: 실차데이터1/0. no_pothole/1240.png  

In [ ]:
# 1. 기본 뿌리 디렉터리와 검증데이터 경로를 가져온다
base_path = os.path.abspath('/content/실차데이터1')
print(base_path)

# 2. 데이터 경로를 지정을 해준다.
no_pothole_dir = os.path.join(base_path, '0. no_pothole')
pothole_dir = os.path.join(base_path, '1. pothole')
filled_dir = os.path.join(base_path, '2. filled')

# 3. 데이터 셋의 개수를 카운팅한다.
num_no_pothole = len(os.listdir(no_pothole_dir))
num_pothole_dir = len(os.listdir(pothole_dir))
num_filled_dir = len(os.listdir(filled_dir))

total_data = num_no_pothole + num_pothole_dir + num_filled_dir

print(f'정상도로 : {num_no_pothole} 개 , 팟홀도로 : {num_pothole_dir}, 보수도로 : {num_filled_dir}')

print(f'Total Data 개수 : {total_data}')

/content/실차데이터1
정상도로 : 467 개 , 팟홀도로 : 508, 보수도로 : 826
Total Data 개수 : 1801


In [ ]:
# Train set 과 Validation set 을 나누며, Data Augmentation 작업 또한 추가시킨다.
#11/12 여기 배치 정규화 추가하는 코드 찾으면 넣자
train_image_generator = ImageDataGenerator(rescale=1./255,
      rotation_range=8,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.15)
validation_image_generator = ImageDataGenerator(rescale=1./255,
                                                validation_split=0.15)
batch_size = 8
epochs = 15
IMG_HEIGHT = 320
IMG_WIDTH = 320

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=base_path,
                                                           seed = 42,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='training',
                                                           class_mode='sparse')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=base_path,
                                                           seed = 42,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='validation',
                                                           class_mode='sparse')
train_num = train_data_gen.samples # training set 이 몇개인지
val_num = val_data_gen.samples   # validation set 이 몇개인지

Found 1538 images belonging to 3 classes.
Found 270 images belonging to 3 classes.


# Model Loading

In [ ]:
# Create base model
# input_tensor 는 내가 쓰고 싶은 입력크기
# input_shape 는 모델이 가지는 입력크기를 말하는듯

def build_model(hp):
    base_model = tf.keras.applications.Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=None)

    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
    base_model.trainable = False
    #base_model.trainable = False  # 기존에 false 였음
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=False)
    #x = keras.layers.Dropout(0.2)(x)
    x=keras.layers.GlobalAveragePooling2D()(x)
    
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        x=keras.layers.Dense(# Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "elu"]))(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.25)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model


In [ ]:
# Keras_tuner 가 정상 작동하는지 확인
build_model(kt.HyperParameters())

In [ ]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

tuner.search(train_data_gen,validation_data=val_data_gen, epochs=2)


Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
num_layers        |4                 |?                 
units_0           |416               |?                 
activation        |relu              |?                 
dropout           |True              |?                 
lr                |0.0046255         |?                 
units_1           |32                |?                 
units_2           |32                |?                 
units_3           |32                |?                 

Epoch 1/2
193/193 [==============================] - 156s 793ms/step - loss: 0.9177 - accuracy: 0.5195 - val_loss: 0.9353 - val_accuracy: 0.3926
Epoch 2/2
193/193 [==============================] - 152s 788ms/step - loss: 0.6728 - accuracy: 0.6756 - val_loss: 0.7886 - val_accuracy: 0.5148
Epoch 1/2
183/193 [===========================>..] - ETA: 7s - loss: 0.9848 - accuracy: 0.4815

KeyboardInterrupt: ignored